In [ ]:
%sql
CREATE SCHEMA IF NOT EXIST bronze
COMMENT 'Bronze Layer'
LOCALTION 's3://fiap-challenge/databricks/bronze';

CREATE TABLE bronze.health (

);

CREATE TABLE bronze.sales (

);

In [ ]:
%python
from pyspark.sql import SparkSession

checkpoint_location_health = 's3://fiap-challenge/databricks/checkpoint/bronze/health'
checkpoint_location_sales = 's3://fiap-challenge/databricks/checkpoint/bronze/sales'

path_health = 's3://fiap-challenge/databricks/landing/health'
path_sales = 's3://fiap-challenge/databricks/landing/sales'

read_options_health = {'sep': ';', 'header': False}
read_options_sales = {'sep': ';', 'header': False}

spark = SparkSession \
    .builder \
    .config('spark.hadoop.hive.exec.dynamic.partition', 'true') \
    .config('spark.hadoop.hive.exec.dynamic.partition.mode', 'nonstrict') \
    .config('spark.sql.sources.partitionOverwriteMode', 'dynamic') \
    .enableHiveSupport() \
    .appName('fiap-challenge') \
    .getOrCreate()

In [ ]:
%python
df_health = spark \
    .readStream \
    .format('cloudFiles') \
    .option('cloudFiles.format', 'csv') \
    .option(**read_options_health) \
    .load(path_health)

In [ ]:
%python
df_sales = spark \
    .readStream \
    .format('cloudFiles') \
    .option('cloudFiles.format', 'csv') \
    .option(**read_options_sales) \
    .load(path_sales)

In [ ]:
%python
stream_health = df_health \
    .writeStream \
    .format('delta') \
    .option('checkpointLocation', checkpoint_location_health) \
    .trigger(once=True)

stream_health \
    .toTable(tableName='bronze.health')

stream_health.awaitTermination()

In [ ]:
%python
stream_sales = df_sales \
    .writeStream \
    .format('delta') \
    .option('checkpointLocation', checkpoint_location_health) \
    .trigger(once=True)

stream_sales \
    .toTable(tableName='bronze.sales')

stream_sales.awaitTermination()